### Data preprocessing

In [1]:
import pandas as pd
import numpy as np
import re
import math
from datetime import datetime

In [2]:
filename = 'Rotten_tomato_.csv'
df_origin = pd.read_csv(filename)
df_origin[:3]

,name,meter_score,audience_score,rating,genre,cast,director,time,studio,runtime,box_office,movie_synopsis
0,Toy Story 3,98,89%,G,"['Animation', 'Comedy', 'Kids & Family', 'Scie...","['Tom Hanks', 'Tim Allen', 'Joan Cusack', 'Ned...",Lee Unkrich,"Jun 18, 2010",Walt Disney Pictures,103 minutes,"$414,984,497","""Toy Story 3"" welcomes Woody, Buzz and the who..."
1,True Grit,96,85%,PG-13 (for some intense sequences of western v...,"['Action & Adventure', 'Drama', 'Western']","['Hailee Steinfeld', 'Jeff Bridges', 'Matt Dam...","Joel Coen, \n \n ...","Dec 22, 2010",Paramount Pictures,110 minutes,"$171,031,347",Fourteen-year-old Mattie Ross (Hailee Steinfel...
2,How to Train Your Dragon,98,91%,PG (for sequences of intense action and some s...,"['Animation', 'Kids & Family', 'Science Fictio...","['Jay Baruchel', 'Gerard Butler', 'Craig Fergu...","Dean DeBlois, Chris Sanders","Mar 26, 2010",Paramount/DWA,98 minutes,"$216,900,000",The son of a Viking chief must capture a drago...


#### examine null value

In [3]:
df = df_origin.dropna(axis=0,how='all')
len(df) / len(df_origin)

1.0

#### examine and convert data types

In [4]:
print(df['meter_score'].describe())
# meter_score is in integer type
# no outliner

count    1735.000000
mean       88.605187
std         7.329474
min        57.000000
25%        84.000000
50%        90.000000
75%        94.000000
max       100.000000
Name: meter_score, dtype: float64


In [5]:
print(df['audience_score'].describe())
# audience_score is in str type

# convert audience_score from str into float for convience
df['audience_score'] = df['audience_score'].apply(lambda x: int(x[:-1]))

count     1735
unique      62
top        86%
freq        84
Name: audience_score, dtype: object


#### set rating column as categorical data

In [6]:
# set ratting column as categorical data, delete the explanation of genre in the braket
def set_rating(pattern, string):
    res = re.match(pattern,string)
    return res.group()

pattern = r'[\w-]+'
df['rating'] = df['rating'].apply(lambda x: set_rating(pattern,x))

#### set genre as a list of genres

In [7]:
# convert genre from str into list
def set_genre(string):
    l = string[1:-1].strip().split(', ')
    res = [x[1:-1] for x in l]
    return res

df['genre'] = df['genre'].apply(lambda x: set_genre(x))

#### set cast as a list of casts

In [8]:
# convert column from str into list
def set_cast(string):
    try:
        if math.isnan(string):
            return []
    except:
        l = string[1:-1].strip().split(', ')
        res = [x[1:-1] for x in l]
        return res

df['cast'] = df['cast'].apply(lambda x: set_cast(x))

#### clean cast data

In [9]:
# delete non-name elements in the list
df['cast'] = df['cast'].apply(lambda x: x[:-1])

#### clean director column

In [10]:
# convert director from str into list
# clean unformatted data
def set_director(string):
    try:
        if math.isnan(string):
            return []
    except:
        res = string.strip().split(', ')
        pattern = r'[\s]+ ([A-Za-z ]+)'
        for ele in res:
            if len(ele)>50:
                res.remove(ele)
                match = re.match(pattern, ele)
                res.append(match.groups()[0])
    return res

df['director'] = df['director'].apply(lambda x: set_director(x))

#### reset time column

In [11]:
# convert time from str into datetime type
# clean unformatted records
def set_date(string):
    try:
        if math.isnan(string):
            return None
    except:
        if len(string)<40:
            return datetime.strptime(string,"%b %d, %Y")
        else:
            return None

df['time'] = df['time'].apply(lambda x: set_date(x))

#### drop empty time records

In [12]:
print(len(df[df['time'].isnull()]))
df = df[~df['time'].isnull()]

23


#### clean studio column

In [13]:
# convert studio from str into list
# clean unformatted records
df['studio'] = df['studio'].apply(lambda x: x.strip().split('/'))

In [14]:
# some studio names are abbreviations
# be careful!
print(df['studio'][1])
print(df['studio'][2])

['Paramount Pictures']
['Paramount', 'DWA']


#### reset runtime column

In [15]:
df['runtime'] = df['runtime'].apply(lambda x: x[:-8])

#### reset boxoffice column

In [16]:
# clean unformatted boxoffice records
def set_boxoffice(string):
    try:
        if string[0]=='$':
            return int(string[1:].replace(',',''))
    except:
        return np.NaN

df['box_office'] = df['box_office'].apply(lambda x: set_boxoffice(x))

In [17]:
# delete NaN boxoffice records
df = df[~df['box_office'].isnull()]

### preprocessed data

In [20]:
df[:5]

,name,meter_score,audience_score,rating,genre,cast,director,time,studio,runtime,box_office,movie_synopsis
0,Toy Story 3,98,89,G,"[Animation, Comedy, Kids & Family, Science Fic...","[Tom Hanks, Tim Allen, Joan Cusack, Ned Beatty...",[Lee Unkrich],2010-06-18,[Walt Disney Pictures],103,414984497.0,"""Toy Story 3"" welcomes Woody, Buzz and the who..."
1,True Grit,96,85,PG-13,"[Action & Adventure, Drama, Western]","[Hailee Steinfeld, Jeff Bridges, Matt Damon, J...","[Joel Coen, Ethan Coen]",2010-12-22,[Paramount Pictures],110,171031347.0,Fourteen-year-old Mattie Ross (Hailee Steinfel...
2,How to Train Your Dragon,98,91,PG,"[Animation, Kids & Family, Science Fiction & F...","[Jay Baruchel, Gerard Butler, Craig Ferguson, ...","[Dean DeBlois, Chris Sanders]",2010-03-26,"[Paramount, DWA]",98,216900000.0,The son of a Viking chief must capture a drago...
3,The King's Speech,95,92,PG-13,[Drama],"[Colin Firth, Helena Bonham Carter, Geoffrey R...",[Tom Hooper],2010-11-26,[The Weinstein Company],118,138300000.0,After the death of his father King George V (M...
4,Inside Job,98,91,PG-13,"[Documentary, Special Interest]","[Matt Damon, William Ackman, Daniel Alpert, Jo...",[Charles Ferguson (III)],2010-10-08,[Sony Pictures Classics],108,4311834.0,"From Academy Award (R) nominated filmmaker, Ch..."


In [27]:
df.to_csv('data.csv',index=True,sep=',')

['Lee Unkrich']